In [1470]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge,TweedieRegressor
from lightgbm import LGBMClassifier
#import tensorflow
from sklearn.ensemble import AdaBoostRegressor 
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import time
from tabpfn import TabPFNRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif

#### Initial Work on Data

In [1472]:
df = pd.read_csv("mw_pw_profiles.csv")
df2 = pd.read_csv("mw_overall.csv")

/var/folders/s8/l066lscn1192jjqlcbvxsyz40000gn/T/ipykernel_76793/2062031687.py:1: DtypeWarning: Columns (2,33,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("mw_pw_profiles.csv")


In [1473]:
cutoff_date = '2022-06-06'

In [1474]:
df.shape

(409559, 56)

In [1475]:
df.columns

Index(['Unnamed: 0', 'player_id', 'match_id', 'gender', 'balls_per_over',
       'start_date', 'series_name', 'match_type', 'name_x', 'runs_scored',
       'player_out', 'balls_faced', 'fours_scored', 'sixes_scored',
       'catches_taken', 'run_out_direct', 'run_out_throw', 'stumpings_done',
       'out_kind', 'dot_balls_as_batsman', 'order_seen', 'balls_bowled',
       'runs_conceded', 'wickets_taken', 'bowled_done', 'lbw_done', 'maidens',
       'dot_balls_as_bowler', 'player_team', 'opposition_team', 'name_y',
       'unique_name', 'key_bcci', 'key_bcci_2', 'key_bigbash', 'key_cricbuzz',
       'key_cricheroes', 'key_crichq', 'key_cricinfo', 'key_cricinfo_2',
       'key_cricinfo_3', 'key_cricingif', 'key_cricketarchive',
       'key_cricketarchive_2', 'key_cricketworld', 'key_nvplay',
       'key_nvplay_2', 'key_opta', 'key_opta_2', 'key_pulse', 'key_pulse_2',
       'full_name', 'batting_style', 'bowling_style', 'playing_role', 'teams'],
      dtype='object')

In [1476]:
df2.shape

(40814, 45)

In [1477]:
df2.columns

Index(['match_id', 'innings', 'batting_team', 'runs_off_bat', 'extras',
       'wides', 'noballs', 'byes', 'legbyes', 'penalty', 'player_dismissed',
       'bowled', 'caught', 'caught and bowled', 'lbw', 'run out',
       'balls_per_over', 'city', 'dates', 'event_name', 'match_number',
       'gender', 'match_type', 'match_type_number', 'match_referees',
       'tv_umpires', 'umpires', 'result', 'player_of_match', 'season',
       'team_type', 'teams', 'toss_decision', 'toss_winner', 'venue', 'winner',
       'players', 'stumped', 'hit wicket', 'retired hurt', 'retired not out',
       'obstructing the field', 'retired out', 'handled the ball',
       'hit the ball twice'],
      dtype='object')

In [1478]:
df2['gender'].value_counts()

gender
male      34118
female     6696
Name: count, dtype: int64

In [1479]:
def add_stadium(d1, d2):
    # Create a dictionary mapping match_id to the last venue
    venue_map = d2.groupby('match_id')['venue'].last().to_dict()

    # Map match_id in d1 to stadium values using the dictionary
    d1['stadium'] = d1['match_id'].map(venue_map)

    # Factorize the 'stadium' column for numerical encoding
    d1['stadium'], _ = pd.factorize(d1['stadium'])

    return d1

In [1480]:
df = add_stadium(df, df2)

In [1481]:
len(df)

409559

In [1482]:
df['stadium'].value_counts()

stadium
-1      16384
 62      7175
 18      7164
 330     5413
 156     4649
        ...  
 800       20
 762       20
 819       19
 824       19
 720       18
Name: count, Length: 828, dtype: int64

In [1483]:
df['match_type'].value_counts()

match_type
T20     248312
ODI      63628
MDM      37952
ODM      33708
Test     18919
IT20      7040
Name: count, dtype: int64

In [1484]:
ODM = df[df['match_type'] == 'ODM']

In [1485]:
ODM[ODM['player_team'] == 'India'].shape

(44, 57)

In [1486]:
ODM[ODM['player_team'] == 'Australia'].shape

(0, 57)

In [1487]:
ODI = df[df['match_type'] == 'ODI']

In [1488]:
ODI['player_team'].unique()

array(['Australia', 'Bangladesh', 'Scotland', 'Africa XI', 'Kenya',
       'India', 'Sri Lanka', 'Papua New Guinea', 'South Africa',
       'Namibia', 'ICC World XI', 'England', 'United Arab Emirates',
       'Netherlands', 'Hong Kong', 'Zimbabwe', 'New Zealand', 'Pakistan',
       'West Indies', 'Bermuda', 'Ireland', 'Canada', 'Oman', 'Thailand',
       'United States of America', 'Jersey', 'Asia XI', 'Nepal'],
      dtype=object)

In [1489]:
ODI = ODI[
    (ODI['player_team'].isin(['Australia', 'India', 'Bangladesh', 'England', 'South Africa', 'Pakistan', 'Afghanistan', 'New Zealand'])) & 
    (ODI['gender'] == 'male')
]

In [1490]:
ODI.shape

(31320, 57)

In [1491]:
df[df['player_team'] == 'Afghanistan']
## public data mein afg nahi hai, ig isiliye hoga

,Unnamed: 0,player_id,match_id,gender,balls_per_over,start_date,series_name,match_type,name_x,runs_scored,...,key_opta,key_opta_2,key_pulse,key_pulse_2,full_name,batting_style,bowling_style,playing_role,teams,stadium


In [1492]:
T20 = df[df['match_type'] == 'T20']
T20['player_team'].unique()

array(['Sierra Leone', 'Malawi', 'Vanuatu', 'Hong Kong', 'Tripura',
       'Brisbane Heat', 'Adelaide Strikers', 'Sylhet Royals', 'Haryana',
       'Gibraltar', 'Australia', 'Trinbago Knight Riders',
       'Trinidad and Tobago', 'Bangladesh', 'Denmark', 'Boland', 'Fiji',
       'Thailand', 'Sunrisers', 'St Lucia Zouks', 'North-West Warriors',
       'Singapore', 'Indonesia', 'Jersey', 'Loughborough Lightning',
       'England', 'Lightning', 'Birmingham Phoenix', 'Otago', 'The Blaze',
       'Trent Rockets', 'Uttarakhand', 'Scotland', 'Central Districts',
       'Cook Islands', 'Botswana', 'Samoa', 'Perth Scorchers',
       'Lancashire', 'Manchester Originals', 'Norway', 'Bahamas',
       'Nottinghamshire', 'Kent', 'Melbourne Stars', 'Gujarat Giants',
       'Kenya', 'Rising Pune Supergiant', 'Karnataka', 'Kings XI Punjab',
       'Punjab Kings', 'Sunrisers Hyderabad',
       'Royal Challengers Bangalore', 'Delhi Daredevils', 'Croatia',
       'Sudur Paschim Royals', 'Cayman Islands', 

In [1493]:
T20 = T20[
    (T20['player_team'].isin(['Australia', 'India', 'Bangladesh', 'England', 'South Africa', 'Pakistan', 'Afghanistan', 'New Zealand'])) & 
    (T20['gender'] == 'male')
]

In [1494]:
T20.shape

(16097, 57)

In [1495]:
it20 = df[df['match_type'] == 'IT20']
it20=it20[
    (it20['player_team'].isin(['Australia', 'India', 'Bangladesh', 'England', 'South Africa', 'Pakistan', 'Afghanistan', 'New Zealand'])) & 
    (it20['gender'] == 'male')
]

In [1496]:
T20 = pd.concat([T20,it20])
T20 = T20.reset_index(drop=True)

In [1497]:
test = df[df['match_type'] == 'Test']
test['player_team'].unique()

array(['Australia', 'England', 'India', 'Sri Lanka', 'ICC World XI',
       'Zimbabwe', 'Bangladesh', 'New Zealand', 'South Africa',
       'Pakistan', 'Ireland', 'West Indies'], dtype=object)

In [1498]:
test=test[
    (test['player_team'].isin(['Australia', 'India', 'Bangladesh', 'England', 'South Africa', 'Pakistan', 'Afghanistan', 'New Zealand'])) & 
    (test['gender'] == 'male')
]

In [1499]:
ODI = ODI.sort_values(by='start_date').reset_index(drop=True)
T20 = T20.sort_values(by='start_date').reset_index(drop=True)
test = test.sort_values(by='start_date').reset_index(drop=True)

#### Now I'll add any remaining matches that players have played

In [1501]:
ODI.columns

Index(['Unnamed: 0', 'player_id', 'match_id', 'gender', 'balls_per_over',
       'start_date', 'series_name', 'match_type', 'name_x', 'runs_scored',
       'player_out', 'balls_faced', 'fours_scored', 'sixes_scored',
       'catches_taken', 'run_out_direct', 'run_out_throw', 'stumpings_done',
       'out_kind', 'dot_balls_as_batsman', 'order_seen', 'balls_bowled',
       'runs_conceded', 'wickets_taken', 'bowled_done', 'lbw_done', 'maidens',
       'dot_balls_as_bowler', 'player_team', 'opposition_team', 'name_y',
       'unique_name', 'key_bcci', 'key_bcci_2', 'key_bigbash', 'key_cricbuzz',
       'key_cricheroes', 'key_crichq', 'key_cricinfo', 'key_cricinfo_2',
       'key_cricinfo_3', 'key_cricingif', 'key_cricketarchive',
       'key_cricketarchive_2', 'key_cricketworld', 'key_nvplay',
       'key_nvplay_2', 'key_opta', 'key_opta_2', 'key_pulse', 'key_pulse_2',
       'full_name', 'batting_style', 'bowling_style', 'playing_role', 'teams',
       'stadium'],
      dtype='object')

In [1502]:
players_added = set()

In [1503]:
df['stadium'].value_counts()

stadium
-1      16384
 62      7175
 18      7164
 330     5413
 156     4649
        ...  
 800       20
 762       20
 819       19
 824       19
 720       18
Name: count, Length: 828, dtype: int64

In [1504]:
def add_remaining_matches(useful_data ,data, players_added, players):
    print(f"Initial Shape : {useful_data.shape}")
    new_data = pd.DataFrame(columns = data.columns)

    for player in players:
        if not player in players_added:
            temp = data[(data['player_id'] == player) & ~(data['player_team'].isin(['Australia', 'India', 'Bangladesh', 'England', 'South Africa', 'Pakistan', 'Afghanistan', 'New Zealand']))]
            if not temp.empty:
                new_data = pd.concat([new_data, temp])
            players_added.add(player)
    
    if new_data.empty:
        print(f"Final Shape : {useful_data.shape}")
        return useful_data

    useful_data = pd.concat([useful_data, new_data])
    print(f"Final Shape : {useful_data.shape}")

    return useful_data   

In [ ]:
ODI = add_remaining_matches(ODI, df, players_added, ODI['player_id'].unique())

Initial Shape : (31320, 57)


/var/folders/s8/l066lscn1192jjqlcbvxsyz40000gn/T/ipykernel_76793/3070751945.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_data = pd.concat([new_data, temp])


In [ ]:
T20 = add_remaining_matches(T20, df ,players_added, T20['player_id'].unique())

In [ ]:
test = add_remaining_matches(test, df, players_added, test['player_id'].unique())

In [ ]:
ODI = ODI.sort_values(by='start_date').reset_index(drop=True)
T20 = T20.sort_values(by='start_date').reset_index(drop=True)
test = test.sort_values(by='start_date').reset_index(drop=True)

#### Now we have combined domestic & international data for the players. Now we calculate fantasy scores

In [ ]:
def calculate_fantasy_scores(df):
    df['fantasy_score_batting'] = 0
    df['fantasy_score_bowling'] = 0
    df['fantasy_score_total'] = 0
    for index, row in df.iterrows():
        # Batting fantasy score calculation
        runs_scored = row['runs_scored']
        balls_faced = row['balls_faced']
        fours_scored = row['fours_scored']
        sixes_scored = row['sixes_scored']
        catches_taken = row['catches_taken']
        match_type = row['match_type']
        series = row['series_name']
        player_out = row['player_out']
        stumpings = row['stumpings_done']
        fantasy_playing=0
        fantasy_batting = 0
        if match_type in ['T20', 'IT20']:
            if(match_type == 'T20'):
                fantasy_playing=4
            fantasy_batting = 1*runs_scored + 1*(fours_scored) + 2*(sixes_scored) + 12*stumpings
            if runs_scored >= 100:
                fantasy_batting += 16
            elif runs_scored >= 50:
                fantasy_batting += 8
            elif runs_scored >= 30:
                fantasy_batting += 4

            if runs_scored == 0 and player_out:
                fantasy_batting -= 2

            if balls_faced >= 10:
                strike_rate = (runs_scored / balls_faced) * 100
                if strike_rate < 50:
                    fantasy_batting -= 6
                elif strike_rate < 60:
                    fantasy_batting -= 4
                elif strike_rate <= 70:
                    fantasy_batting -= 2
                elif strike_rate > 170:
                    fantasy_batting += 6
                elif strike_rate > 150:
                    fantasy_batting += 4
                elif strike_rate >= 130:
                    fantasy_batting += 2

        elif match_type in ['ODI', 'ODM']:
            if(match_type == 'ODI'):
                fantasy_playing=4
            fantasy_batting = 1*runs_scored + 1*(fours_scored) + 2*(sixes_scored) + 12*stumpings
            if runs_scored >= 100:
                fantasy_batting += 8
            elif runs_scored >= 50:
                fantasy_batting += 4

            if runs_scored == 0 and player_out:
                fantasy_batting -= 3

            if balls_faced >= 20:
                strike_rate = (runs_scored / balls_faced) * 100
                if strike_rate < 30:
                    fantasy_batting -= 6
                elif strike_rate < 40:
                    fantasy_batting -= 4
                elif strike_rate <= 50:
                    fantasy_batting -= 2
                elif strike_rate > 140:
                    fantasy_batting += 6
                elif strike_rate > 120:
                    fantasy_batting += 4
                elif strike_rate >= 100:
                    fantasy_batting += 2

        elif match_type in ['Test', 'MDM']:
            if(match_type == 'Test'):
                fantasy_playing=4
            fantasy_batting = 1*runs_scored + 1*(fours_scored) + 2*(sixes_scored) + 12*stumpings
            if runs_scored >= 100:
                fantasy_batting += 8
            elif runs_scored >= 50:
                fantasy_batting += 4

            if runs_scored == 0 and player_out:
                fantasy_batting -= 4

        elif match_type == 'T10':
            fantasy_playing=4
            fantasy_batting = 1*runs_scored + 1*(fours_scored) + 2*(sixes_scored) + 12*stumpings
            if runs_scored >= 50:
                fantasy_batting += 16
            elif runs_scored >= 30:
                fantasy_batting += 8

            if runs_scored == 0 and player_out:
                fantasy_batting -= 2

            if balls_faced >= 5:
                strike_rate = (runs_scored / balls_faced) * 100
                if strike_rate < 60:
                    fantasy_batting -= 6
                elif strike_rate < 70:
                    fantasy_batting -= 4
                elif strike_rate <= 80:
                    fantasy_batting -= 2
                elif strike_rate > 190:
                    fantasy_batting += 6
                elif strike_rate > 170:
                    fantasy_batting += 4
                elif strike_rate >= 150:
                    fantasy_batting += 2

        elif series == '6ixty':
            fantasy_playing=4
            fantasy_batting = 1*runs_scored + 1*(fours_scored) + 2*(sixes_scored) + 12*stumpings
            if runs_scored >= 50:
                fantasy_batting += 16
            elif runs_scored >= 30:
                fantasy_batting += 8

            if runs_scored == 0 and player_out:
                fantasy_batting -= 2

            if balls_faced >= 5:
                strike_rate = (runs_scored / balls_faced) * 100
                if strike_rate < 60:
                    fantasy_batting -= 6
                elif strike_rate < 70:
                    fantasy_batting -= 4
                elif strike_rate <= 80:
                    fantasy_batting -= 2
                elif strike_rate > 190:
                    fantasy_batting += 6
                elif strike_rate > 170:
                    fantasy_batting += 4
                elif strike_rate >= 150:
                    fantasy_batting += 2
        elif series == '6ixty':
            fantasy_playing=4
            fantasy_batting = 1*runs_scored + 1*(fours_scored) + 2*(sixes_scored) + 12*stumpings
            if runs_scored >= 50:
                fantasy_batting += 16
            elif runs_scored >= 30:
                fantasy_batting += 8

            if runs_scored == 0 and player_out:
                fantasy_batting -= 2

            if balls_faced >= 5:
                strike_rate = (runs_scored / balls_faced) * 100
                if strike_rate < 60:
                    fantasy_batting -= 6
                elif strike_rate < 70:
                    fantasy_batting -= 4
                elif strike_rate <= 80:
                    fantasy_batting -= 2
                elif strike_rate > 190:
                    fantasy_batting += 6
                elif strike_rate > 170:
                    fantasy_batting += 4
                elif strike_rate >= 150:
                    fantasy_batting += 2
        elif series == 'The100':
            fantasy_playing=4
            fantasy_batting = 1*runs_scored + 1*(fours_scored) + 2*(sixes_scored) + 12*stumpings
            if runs_scored >= 100:
                fantasy_batting += 20
            elif runs_scored >= 50:
                fantasy_batting += 10
            elif runs_scored >= 30:
                fantasy_batting += 5

            if runs_scored == 0 and player_out:
                fantasy_batting -= 2

        df.at[index, 'fantasy_score_batting'] = fantasy_batting

        # Bowling fantasy score calculation
        balls_bowled = row['balls_bowled']
        runs_conceded = row['runs_conceded']
        wickets_taken = row['wickets_taken']
        maidens = row['maidens']
        bowled_done = row['bowled_done']
        lbw_done = row['lbw_done']
        run_out_direct = row['run_out_direct']
        run_out_throw = row['run_out_throw']

        fantasy_bowling = 0

        if match_type in ['T20', 'IT20']:
          fantasy_bowling = 25*(wickets_taken) + 8*(lbw_done+bowled_done) + 12*maidens + 8*catches_taken
          if wickets_taken >= 5:
              fantasy_bowling += 16
          elif wickets_taken >= 4:
              fantasy_bowling += 8
          elif wickets_taken >= 3:
              fantasy_bowling += 4

          if catches_taken >= 3:
              fantasy_bowling += 4

          if balls_bowled >= 12:
              economy_rate = (runs_conceded / balls_bowled) * 6
              if economy_rate < 5:
                  fantasy_bowling += 6
              elif economy_rate < 6:
                  fantasy_bowling += 4
              elif economy_rate <= 7:
                  fantasy_bowling += 2
              elif economy_rate > 12:
                  fantasy_bowling -= 6
              elif economy_rate > 11:
                  fantasy_bowling -= 4
              elif economy_rate >= 10:
                  fantasy_bowling -= 2
          fantasy_bowling += run_out_throw*6 + run_out_direct*12

        elif match_type in ['ODI', 'ODM']:
            fantasy_bowling = 25*(wickets_taken) + 8*(lbw_done+bowled_done) + 4*maidens + 8*catches_taken
            if wickets_taken >= 5:
                fantasy_bowling += 8
            elif wickets_taken >= 4:
                fantasy_bowling += 4

            if catches_taken >= 3:
                fantasy_bowling += 4

            if balls_bowled >= 30:
                economy_rate = (runs_conceded / balls_bowled) * 6
                if economy_rate < 2.5:
                    fantasy_bowling += 6
                elif economy_rate < 3.5:
                    fantasy_bowling += 4
                elif economy_rate <= 4.5:
                    fantasy_bowling += 2
                elif economy_rate > 9:
                    fantasy_bowling -= 6
                elif economy_rate > 8:
                    fantasy_bowling -= 4
                elif economy_rate >= 7:
                    fantasy_bowling -= 2
            fantasy_bowling += run_out_throw*6 + run_out_direct*12

        elif match_type in ['Test', 'MDM']:
            fantasy_bowling = 16*(wickets_taken) + 8*(lbw_done+bowled_done) + 8*catches_taken
            if wickets_taken >= 5:
                fantasy_bowling += 8
            elif wickets_taken >= 4:
                fantasy_bowling += 4
            fantasy_bowling += run_out_throw*6 + run_out_direct*12

        elif match_type == 'T10':
            fantasy_bowling = 25*(wickets_taken) + 8*(lbw_done+bowled_done) + 16*maidens + 8*catches_taken
            if wickets_taken >= 3:
                fantasy_bowling += 16
            elif wickets_taken >= 2:
                fantasy_bowling += 8

            if balls_bowled >= 6:
                economy_rate = (runs_conceded / balls_bowled) * 6
                if economy_rate < 7:
                    fantasy_bowling += 6
                elif economy_rate < 8:
                    fantasy_bowling += 4
                elif economy_rate <= 9:
                    fantasy_bowling += 2
                elif economy_rate > 16:
                    fantasy_bowling -= 6
                elif economy_rate > 15:
                    fantasy_bowling -= 4
                elif economy_rate >= 14:
                    fantasy_bowling -= 2
            fantasy_bowling += run_out_throw*6 + run_out_direct*12

            if catches_taken >= 3:
                fantasy_bowling += 4
        elif match_type == '6ixty':
            fantasy_bowling = 25*(wickets_taken) + 8*(lbw_done+bowled_done) + 16*maidens + 8*catches_taken
            if wickets_taken >= 3:
                fantasy_bowling += 16
            elif wickets_taken >= 2:
                fantasy_bowling += 8

            if balls_bowled >= 6:
                economy_rate = (runs_conceded / balls_bowled) * 6
                if economy_rate < 7:
                    fantasy_bowling += 6
                elif economy_rate < 8:
                    fantasy_bowling += 4
                elif economy_rate <= 9:
                    fantasy_bowling += 2
                elif economy_rate > 16:
                    fantasy_bowling -= 6
                elif economy_rate > 15:
                    fantasy_bowling -= 4
                elif economy_rate >= 14:
                    fantasy_bowling -= 2
            fantasy_bowling += run_out_throw*6 + run_out_direct*12

            if catches_taken >= 3:
                fantasy_bowling += 4

        elif match_type == 'The100':
            fantasy_bowling = 25*(wickets_taken) + 8*(lbw_done+bowled_done) + 8*catches_taken
            if wickets_taken >= 5:
                fantasy_bowling += 20
            elif wickets_taken >= 4:
                fantasy_bowling += 10
            elif wickets_taken >= 3:
                fantasy_bowling += 5
            elif wickets_taken >= 2:
                fantasy_bowling += 3
            fantasy_bowling += run_out_throw*6 + run_out_direct*12

            if catches_taken >= 3:
                fantasy_bowling += 4

        df.at[index, 'fantasy_score_bowling'] = fantasy_bowling
        df.at[index, 'fantasy_score_total'] = fantasy_playing
    df['fantasy_score_total'] = df['fantasy_score_batting'] + df['fantasy_score_bowling']+df['fantasy_score_total']
    return df

In [ ]:
ODI=calculate_fantasy_scores(ODI)
T20=calculate_fantasy_scores(T20)
test=calculate_fantasy_scores(test)

### Feature Generation

In [ ]:
def prepare_data(data, cutoff_date):
    """Prepare and sort data, set date format, and apply cutoff date."""
    data['start_date'] = pd.to_datetime(data['start_date'])  # Ensure 'start_date' is in datetime format
    data = data.sort_values(['player_id', 'match_type', 'start_date'])
    return data[data['start_date'] < cutoff_date]

In [ ]:
def calculate_rolling_fantasy_score(group):
    """Calculate the rolling average of fantasy scores."""
    group['avg_fantasy_score_1'] = group['fantasy_score_total'].shift().rolling(1, min_periods=1).mean()
    group['avg_fantasy_score_5'] = group['fantasy_score_total'].shift().rolling(5, min_periods=1).mean()
    group['avg_fantasy_score_10'] = group['fantasy_score_total'].shift().rolling(10, min_periods=1).mean()
    group['avg_fantasy_score_15'] = group['fantasy_score_total'].shift().rolling(15, min_periods=1).mean()
    group['avg_fantasy_score_20'] = group['fantasy_score_total'].shift().rolling(20, min_periods=1).mean()
    
    return group  # ✅ Ensure function returns the modified DataFrame

In [ ]:
# yaha tak toh no look forward

In [ ]:
def player_features(data, n1,n2,n3, cutoff_date):
    """Main function to calculate player features using helper functions."""
    cutoff_date = pd.to_datetime(cutoff_date)
    data = prepare_data(data, cutoff_date)

    feature_data = []

    for (name, match_type), group in data.groupby(['player_id', 'match_type']):
        group = group.sort_values('start_date')
        group = calculate_rolling_fantasy_score(group)
        feature_data.append(group[['player_id','match_id' , 'start_date', 'avg_fantasy_score_1','avg_fantasy_score_5','avg_fantasy_score_10','avg_fantasy_score_15','avg_fantasy_score_20']])

    result_df = pd.concat(feature_data)
    result_df = result_df.reset_index(drop=True)  # Reset index and drop old index

    return result_df

In [ ]:
# Ensure that 'start_date' columns are in datetime format
ODI['start_date'] = pd.to_datetime(ODI['start_date'])
T20['start_date'] = pd.to_datetime(T20['start_date'])

# Preprocess the T20 data to create a mapping of player_ids to their sorted match dates and fantasy scores
player_t20_data = {}

# Get unique player_ids in T20 data
players_in_t20 = T20['player_id'].unique()

for player_id in players_in_t20:
    # Filter T20 data for the current player
    player_data = T20[T20['player_id'] == player_id].copy()
    # Sort the player's data by 'start_date'
    player_data.sort_values('start_date', inplace=True)
    # Ensure 'start_date' is datetime and no missing values
    player_data = player_data[player_data['start_date'].notnull()]
    # Extract the dates and fantasy scores as pandas Series
    dates = player_data['start_date']
    scores = player_data['fantasy_score_total']
    # Store the Series in a dictionary with player_id as the key
    player_t20_data[player_id] = {'dates': dates, 'scores': scores}

# Define the list of n values
n_values = [1, 5, 10, 15, 20]

# Define a function to compute the rolling mean for each ODI match row
def get_last_n_t20_mean(row, n):
    player_id = row['player_id']
    date = row['start_date']

    # Check if the player exists in T20 data
    if player_id not in player_t20_data:
        return np.nan  # Player has no T20 data; return NaN

    t20_dates = player_t20_data[player_id]['dates']
    t20_scores = player_t20_data[player_id]['scores']

    # Ensure that date is not NaT
    if pd.isna(date):
        return np.nan

    # Find the index where the ODI match date would fit in the player's T20 dates
    idx = t20_dates.searchsorted(date)
    # Determine the indices for the last n matches before the ODI match date
    start_idx = max(0, idx - n)
    end_idx = idx  # up to but not including idx
    scores_to_use = t20_scores.iloc[start_idx:end_idx]

    # Check if there are any matches to compute the mean
    if len(scores_to_use) == 0:
        return np.nan  # No prior matches; return NaN
    else:
        # Compute and return the mean of the last n T20 fantasy scores
        return scores_to_use.mean()

In [ ]:
results = player_features(ODI, 3, 7, 12, cutoff_date)
df = ODI.merge(results, on = ['player_id', 'start_date', 'match_id'], how = 'left')

In [ ]:
num_unique_combinations = ODI[['player_id', 'start_date','match_type','match_id']].drop_duplicates().shape[0]
num_unique_combinations

In [ ]:
results.shape

In [ ]:
results.columns

In [ ]:
for n in n_values:
    col_name = f'last_{n}_t20_mean'
    print(f"Calculating rolling mean for n={n}")
    df[col_name] = df.apply(get_last_n_t20_mean, axis=1, args=(n,))

In [ ]:
# Ensure that 'start_date' columns are in datetime format
ODI['start_date'] = pd.to_datetime(ODI['start_date'])
test['start_date'] = pd.to_datetime(test['start_date'])

# Preprocess the test data to create a mapping of player_ids to their sorted match dates and fantasy scores
player_test_data = {}

# Get unique player_ids in test data
players_in_test = test['player_id'].unique()

for player_id in players_in_test:
    # Filter test data for the current player
    player_data = test[test['player_id'] == player_id].copy()
    # Sort the player's data by 'start_date'
    player_data.sort_values('start_date', inplace=True)
    # Ensure 'start_date' is datetime and has no missing values
    player_data = player_data[player_data['start_date'].notnull()]
    # Extract the dates and fantasy scores as pandas Series
    dates = player_data['start_date']
    scores = player_data['fantasy_score_total']
    # Store the Series in a dictionary with player_id as the key
    player_test_data[player_id] = {'dates': dates, 'scores': scores}

# Define the list of n values
n_values = [1, 5, 10, 15, 20]

# Define a function to compute the rolling mean for each ODI match row
def get_last_n_test_mean(row, n):
    player_id = row['player_id']
    date = row['start_date']
    
    # Check if the player exists in test data
    if player_id not in player_test_data:
        return np.nan  # Player has no test data; return NaN
    
    test_dates = player_test_data[player_id]['dates']
    test_scores = player_test_data[player_id]['scores']
    
    # Ensure that date is not NaT
    if pd.isna(date):
        return np.nan
    
    # Find the index where the ODI match date would fit in the player's test dates
    idx = test_dates.searchsorted(date)
    # Determine the indices for the last n matches before the ODI match date
    start_idx = max(0, idx - n)
    end_idx = idx  # up to but not including idx
    scores_to_use = test_scores.iloc[start_idx:end_idx]
    
    # Check if there are any matches to compute the mean
    if len(scores_to_use) == 0:
        return np.nan  # No prior matches; return NaN
    else:
        # Compute and return the mean of the last n test fantasy scores
        return scores_to_use.mean()

# Apply the function for each n and add the results to the ODI dataframe
for n in n_values:
    col_name = f'last_{n}_test_mean'
    print(f"Calculating rolling mean for n={n}")
    df[col_name] = df.apply(get_last_n_test_mean, axis=1, args=(n,))

In [ ]:
df.tail()

In [ ]:
df['num_matches_played'] = df.groupby('player_id').cumcount() + 1
df['num_matches_played'] = df.groupby('player_id')['num_matches_played'].shift(1).fillna(0).astype(int)

In [ ]:
# Ensure 'start_date' is in datetime format
df['start_date'] = pd.to_datetime(df['start_date'])

# Sort data to ensure correct time difference calculation
df = df.sort_values(by=['player_id', 'start_date'])

# Calculate duration since last match
df['days_since_last_match'] = df.groupby('player_id')['start_date'].diff().dt.days

# Fill NaN values (for the first match of each player) with a default value, e.g., -1 or 0
df['days_since_last_match'] = df['days_since_last_match'].fillna(-1).astype(int)

In [ ]:
d_f=pd.read_csv("mw_overall.csv")

In [ ]:
# Convert d_f to dictionary for quick lookup
toss_dict = d_f.set_index("match_id")["toss_decision"].to_dict()

# Function to assign bat_order
def get_bat_order(match_id):
    if match_id in toss_dict:
        return 1 if toss_dict[match_id] == "bat" else 0
    else:
        return 0 

# Apply function to create bat_order column
df["bat_order"] = df["match_id"].apply(get_bat_order)


In [ ]:
df["bat_order"].value_counts()

In [ ]:
df = df.sort_values(by='start_date').reset_index(drop=True)
df.columns

In [ ]:
cols=['fantasy_score_total', 'avg_fantasy_score_1',
       'avg_fantasy_score_5', 'avg_fantasy_score_10', 'avg_fantasy_score_15',
       'avg_fantasy_score_20', 'match_id','player_id','last_1_t20_mean', 'last_5_t20_mean',
       'last_10_t20_mean', 'last_15_t20_mean', 'last_20_t20_mean','last_1_test_mean', 'last_5_test_mean', 'last_10_test_mean',
       'last_15_test_mean', 'last_20_test_mean', 'num_matches_played', 'days_since_last_match',
       'bat_order']

In [ ]:
def split(df, date):
    train = df[df['start_date'] <= pd.to_datetime(date)]
    test = df[df['start_date'] > pd.to_datetime(date)]
    return train,test

In [ ]:
odi = df[df['match_type'] == 'ODI']

In [ ]:
odi_train, odi_test = split(odi, cutoff_date)

In [ ]:
train,test = split(df, cutoff_date)
train = train[cols]
test = test[cols]

In [ ]:
odi_test = odi_test[cols]

In [ ]:
cols = ['fantasy_score_total', 'avg_fantasy_score_1',
       'avg_fantasy_score_5', 'avg_fantasy_score_10', 'avg_fantasy_score_15',
       'avg_fantasy_score_20','last_1_t20_mean','last_5_t20_mean',
       'last_10_t20_mean', 'last_15_t20_mean', 'last_20_t20_mean','last_1_test_mean', 'last_5_test_mean', 'last_10_test_mean',
       'last_15_test_mean', 'last_20_test_mean', 'num_matches_played', 'days_since_last_match',
       'bat_order']
X_train = train[cols]
# X_test = test[cols]

In [ ]:
X_test = odi_test[cols]

### Training Section

In [ ]:
class EnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, models=None, weights=None):
        self.models = models if models is not None else []
        self.weights = weights if weights is not None else [1 / len(self.models)] * len(self.models)

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
        return self

    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        weighted_preds = np.average(predictions, axis=0, weights=self.weights)
        return weighted_preds

def train_models(X_train, y_train):
    models = {
        "xgboost regressor": XGBRegressor(random_state=42),
        "decision tree regressor": DecisionTreeRegressor(random_state=42),
        "linear regression": LinearRegression(),
        "adaboost regressor": AdaBoostRegressor(random_state=42),
        "catboost regressor": CatBoostRegressor(random_state=42, silent=True),
        "bayesian linear regression": BayesianRidge(),
        "lightgbm regressor": LGBMRegressor(boosting_type='gbdt', is_unbalance=True, learning_rate=0.12875, num_leaves=10, random_state=42)
    }
    
    # Add ensemble model combining Linear Regression and LightGBM Regressor
    ensemble_model = EnsembleRegressor(
        models=[LinearRegression(), LGBMRegressor(boosting_type='gbdt', is_unbalance=True, learning_rate=0.12875, num_leaves=10, random_state=42)],
        weights=[0.5, 0.5]
    )
    models["ensemble (linear regression + lightgbm)"] = ensemble_model

    trained_models = {}

    # Reshape X_train and y_train to ensure proper dimensions for each model
    if len(X_train.shape) == 1:  # If X_train is a 1D array, convert it to 2D
        X_train = X_train.reshape(-1, 1)

    if len(y_train.shape) == 1:  # If y_train is a 1D array, no reshaping needed
        pass
    elif len(y_train.shape) == 2:  # If y_train is 2D, flatten it
        y_train = y_train.ravel()

    for name, model in models.items():
        # Train the model
        print(f"Training {name}...")
        model.fit(X_train, y_train)
        trained_models[name] = {
            'model': model,
        }

    return trained_models


In [ ]:
X_train

In [ ]:
y_train = X_train['fantasy_score_total']
y_test = X_test['fantasy_score_total']
X_train.drop('fantasy_score_total',axis = 1, inplace = True)
X_test.drop('fantasy_score_total',axis = 1, inplace = True)

In [ ]:
X_train.isnull().sum()

In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [ ]:
trained_models = train_models(X_train, y_train)

In [ ]:
def predict_scores(trained_models, X_train, X_test):
    # Ensure columns of X_test align with X_train columns
    X_test = X_test[X_train.columns]

    test_data = pd.DataFrame()

    # Loop through each model to predict scores
    for model_name, model_info in trained_models.items():
        model = model_info['model']  # Extract the trained model
        pred_scores = model.predict(X_test)  # Predict the scores

        # Store each model's predicted scores in the DataFrame
        test_data[model_name + '_predicted_score'] = pred_scores

    return test_data

In [ ]:
lin, lgbm = trained_models['linear regression']['model'], trained_models['lightgbm regressor']['model']

In [ ]:
np.corrcoef(lin.predict(X_test), lgbm.predict(X_test))

In [ ]:
def predictions_per_match(trained_models, X_train, X_test, test):
    # Call predict_scores to get the predicted scores DataFrame
    predictions = predict_scores(trained_models, X_train, X_test)

    # Reset indices of test and predictions for alignment
    test_reset = test.reset_index(drop=True)
    predictions = predictions.reset_index(drop=True)

    # Assign match_id and fantasy_score_total from test to predictions DataFrame
    predictions['match_id'] = test_reset.get('match_id')
    predictions['fantasy_score_total'] = test_reset.get('fantasy_score_total')

    return predictions, test_reset

In [ ]:
X_train.columns

In [ ]:
test.columns

In [ ]:
predictions,test_reset=predictions_per_match(trained_models,X_train,X_test,test)

In [ ]:
def avg_percentage_error_per_model(predictions):
    percentage_error_per_model = {model_name: 0 for model_name in predictions.columns if '_predicted_score' in model_name}
    num_matches = predictions['match_id'].nunique() 

    for match_id in predictions['match_id'].unique():
        match_data = predictions[predictions['match_id'] == match_id]
        match_data_sorted = match_data.sort_values(
            by=[col for col in match_data.columns if '_predicted_score' in col], ascending=False
        )
        for model_name in percentage_error_per_model:
            match_data_sorted_model = match_data_sorted.sort_values(by='fantasy_score_total', ascending=False)
            top_11_true = match_data_sorted_model['fantasy_score_total'].iloc[:11].copy()
            match_data_sorted_model = match_data.sort_values(by=model_name, ascending=False)
            if len(top_11_true) > 0:
                top_11_true.iloc[0] *= 2 
            if len(top_11_true) > 1:
                top_11_true.iloc[1] *= 1.5 

            top_11_predicted = match_data_sorted_model['fantasy_score_total'].iloc[:11].copy()
            if len(top_11_predicted) > 0:
                top_11_predicted.iloc[0] *= 2
            if len(top_11_predicted) > 1:
                top_11_predicted.iloc[1] *= 1.5  
            top_11_sum_true = top_11_true.sum()
            top_11_sum_predicted = top_11_predicted.sum()
            if top_11_sum_true != 0:  
                percentage_error = abs(top_11_sum_predicted - top_11_sum_true) / top_11_sum_true * 100
            else:
                percentage_error = 0 

            percentage_error_per_model[model_name] += percentage_error

    avg_percentage_error_per_model = {model_name: total_error / num_matches for model_name, total_error in percentage_error_per_model.items()} if num_matches != 0 else 69

    return avg_percentage_error_per_model

In [ ]:
predictions

In [ ]:
avg_percentage_error_per_model(predictions)

In [ ]:
pip install shap

In [ ]:
import shap
import xgboost as xgb

def explain_model_with_shap(trained_models, X_train, y_train):
    # Loop through each model to compute SHAP values
    for model_name, model_info in trained_models.items():
        model = model_info['model']  # Extract the trained model
        model.fit(X_train, y_train)  # Train the model if needed

        # Check if the model is XGBRegressor or LinearRegression for shap
        if isinstance(model, xgb.XGBRegressor):
            explainer = shap.Explainer(model)
        else:
            explainer = shap.KernelExplainer(model.predict, X_train)  # For models like LinearRegression

        shap_values = explainer.shap_values(X_train)

        # Plot the SHAP values for all features
        shap.summary_plot(shap_values, X_train, max_display=X_train.shape[1])

        break 

# Example call
explain_model_with_shap(trained_models, X_train, y_train)

### Post Model Work

In [ ]:
df.columns

In [ ]:
def return_15_best_players(players, data, trained_models, cols):

    scores = {}

    for player in players:
        model = trained_models['linear regression']['model']
        player_data = pd.concat([
            data[data['full_name'] == player],
            data[data['unique_name'] == player],
            data[data['name_y'] == player]
        ])


        if player_data.empty:
            continue

        test_data = player_data.iloc[-1][cols].drop('fantasy_score_total', errors='ignore')

        # Fill NaN values with 0
        test_data = test_data.fillna(0)

        # Convert to DataFrame with correct feature names
        test_data_df = pd.DataFrame([test_data], columns=test_data.index)

        pred_score = model.predict(test_data_df)[0]  # Predicted score
        scores[player] = pred_score

    top_15_players = [player for player, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:22]]

    return top_15_players, scores


In [ ]:
uni = ['Rohit Gurunath Sharma', 'Shreyas Santosh Iyer', 'Shubman Gill', 'Virat Kohli', 'Yashasvi Bhupendra Kumar Jaiswal', 'Axar Rajeshbhai Patel', 'Hardik Himanshu Pandya', 'Ravindrasinh Anirudhsinh Jadeja', 'Washington Sundar', 'KL Rahul', 'Rishabh Rajendra Pant', 'Arshdeep Singh', 'Harshit Rana', 'Jasprit Jasbirsingh Bumrah', 'Kuldeep Yadav', 'Mohammed Shami', 'Varun Chakravarthy Vinod',
       'Ben Matthew Duckett', 'Henry James Hamilton Brookes', 'Joseph Edward Root', 'Jacob Graham Bethell', 'Jamie Overton', 'Liam Stephen Livingstone', 'Jamie Luke Smith', 'Joseph Charles Buttler', 'Philip Dean Salt', 'Adil Usman Rashid', 'Brydon Alexander Carse', 'Angus Alexander Patrick Atkinson', 'Jofra Chioke Archer', 'Mark Andrew Wood', 'Saqib Mahmood']

In [ ]:
# uni = uni[:31]

In [ ]:
X_train.head()

In [ ]:
top15, sco = return_15_best_players(uni, df.copy(), trained_models, cols)

In [ ]:
top15

In [ ]:
df[df['full_name'] == 'Saqib Mahmood']

### The weighted optimisation for 100 points limit  